<a href="https://colab.research.google.com/github/Hau381/Automated_delivery_robots/blob/main/Train_mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

%cd drive/MyDrive/Data

In [ ]:

import os
import glob
import random
import shutil

raw_data_path = "/content/drive/MyDrive/Data/images"
train_data_path = "/content/drive/MyDrive/Data/split_data/train"
test_data_path = "/content/drive/MyDrive/Data/split_data/test"

if not os.path.exists("/content/drive/MyDrive/Data/split_data"):
  os.mkdir("/content/drive/MyDrive/Data/split_data")
  
try:
  shutil.rmtree(train_data_path)
  shutil.rmtree(test_data_path)
except:
  pass

os.mkdir(train_data_path)
os.mkdir(test_data_path)

total_files = glob.glob(raw_data_path + "/*.png")
print("Sample file = ", total_files[0])
print("Total file = ", len(total_files))

indices = list(range(len(total_files)))
train_indices = random.sample(indices, k = int(len(total_files)*0.9))
print("Total train file = ", len(train_indices))

for i in indices:
  try:
      if (i in train_indices):
        # Copy to train
        shutil.copy(total_files[i], train_data_path + total_files[i].replace(raw_data_path,""))
        shutil.copy(total_files[i].replace("images","labels").replace(".png",".xml"),
                    train_data_path + total_files[i].replace(raw_data_path, "").replace(".png",".xml"))
      else:
        # Copy to test
        
          shutil.copy(total_files[i], test_data_path + total_files[i].replace(raw_data_path, ""))
          shutil.copy(total_files[i].replace("images", "labels").replace(".png", ".xml"),
                    test_data_path + total_files[i].replace(raw_data_path, "").replace(".png",".xml"))
  except:
      os.remove(total_files[i])
      print (total_files[i])

Sample file =  /content/drive/MyDrive/Data/images/1_202.png
Total file =  340
Total train file =  306


In [ ]:
# 3.Convert XML label sang CSV. File CSV lưu trong thư mục split_data
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

for directory in ['train','test']:
        image_path = os.path.join('/content/drive/MyDrive/Data/split_data/{}'.format(directory))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('/content/drive/MyDrive/Data/split_data/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')

Successfully converted xml to csv.
Successfully converted xml to csv.


In [ ]:
# 4. Tải TF Object Detection APi vào thư mục models
%cd /content/drive/MyDrive
!git clone https://github.com/tensorflow/models.git

/content/drive/MyDrive
Cloning into 'models'...
remote: Enumerating objects: 81524, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 81524 (delta 170), reused 299 (delta 151), pack-reused 81194
Receiving objects: 100% (81524/81524), 596.24 MiB | 13.23 MiB/s, done.
Resolving deltas: 100% (58094/58094), done.
Updating files: 100% (3351/3351), done.


In [ ]:
# 5. Compile the protos -  
# Protocol Buffers (Protobuf) is a free and open-source cross-platform data format used to serialize structured data.
%cd /content/drive/MyDrive/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/MyDrive/models/research


In [ ]:
# 6. Cài dặt API
%cd /content/drive/MyDrive/models/research
!cp object_detection/packages/tf2/setup.py . 
!python -m pip install .

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
# 8. CSV to TFRecord. With help from MiAI git hub
%cd /content/drive/MyDrive/models/research
!git clone https://github.com/thangnch/MiAI_SSD_TF2
!cp MiAI_SSD_TF2/generate_tfrecord.py .
!cp MiAI_SSD_TF2/label_map.txt /content/drive/MyDrive/Data

if not os.path.exists("/content/drive/MyDrive/Data/tfrecord_data/"):
  os.mkdir("/content/drive/MyDrive/Data/tfrecord_data/")

!python generate_tfrecord.py --image_dir=/content/drive/MyDrive/Data/split_data/train --csv_input=/content/drive/MyDrive/Data/split_data/train_labels.csv --output_path=/content/drive/MyDrive/Data/tfrecord_data/train.record 
!python generate_tfrecord.py --image_dir=/content/drive/MyDrive/Data/split_data/test --csv_input=/content/drive/MyDrive/Data/split_data/test_labels.csv --output_path=/content/drive/MyDrive/Data/tfrecord_data/test.record 

 

In [ ]:
# 9. Download pretrain and config
if not os.path.exists("/content/drive/MyDrive/pretrained"):
  os.mkdir("/content/drive/MyDrive/pretrained")

%cd /content/drive/MyDrive/pretrained
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
# Unzip
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz


In [ ]:
# Upload label file object-detection.txt to data folder
%cd /content/drive/MyDrive/models
!pip install -r /content/drive/MyDrive/models/research/MiAI_SSD_TF2/setup.txt
!pip install --upgrade opencv-python

In [ ]:
%cd /content/drive/MyDrive/models/research
%cp /content/drive/MyDrive/models/research/object_detection/model_main_tf2.py .
#train 
!python model_main_tf2.py \
--pipeline_config_path=/content/drive/MyDrive/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
--model_dir=/content/gdrive/MyDrive/output_model --alsologtostderr --num_train_steps=5000
     
